#  DATA 364 Final Project Part 2 - Automatic Stock Analysis Engine

## Introduction

Our project aims to develop an automated stock trading algorithm that leverages key technical indicators, specifically Relative Strength Index (RSI), momentum, and moving average crossovers. This algorithm will provide traders with an efficient tool to make informed buy and sell decisions in financial markets. By combining these indicators, our system seeks to capture valuable trading opportunities and enhance portfolio performance.


## Possible Data Sources

We initially looked at several different methods of retreiving our data, including web scraping, pulling from premade datasets, and using various APIs. We quickly ruled out using premade datasets because we wanted to pull current stock information so our tool would be useful in analyzing stocks in the current market conditions. We also tried using web scraping originally on Yahoo Finance and Google Finance but ran into issues with both platforms. Yahoo Finance used to be a great solution for scraping data since it was formatted in HTML that was fast and easy to parse using libraries like Beautiful Soup. However, since last year, they changed their platform to make it very challenging to scrape information, requireing the use of an application like Selenium. 

Similarly, Google Finance is very sensitive to web scraping and only allows requests every 60 seconds, limiting the amount of stocks you can analyze and making the overall user experience unsatisfactory. We eneded up using Yahoo Finance's API, which provides quick and up-to-date data in a dataframe that is ready for analysis. This method allows us to retrieve the most current data so the user will always get the most current information for their analysis. 

We are also using the Alpha Vantage API to retrieve real-time stock information when the market is open for additional information throughout the day. Finally, we are using the News API (NewsAPI.org) that will search for the top news articles for the given company (based on the stock ticker symbol) and display the top five articles for the user to read and gain better awareness of how the company is performing any pertinent inforamtion that may be affecting the stock price. 

## Extract Data

In [ ]:
#Import packages
import streamlit as st
from datetime import date, time, datetime
import yfinance as yf
from prophet import Prophet
import requests
from prophet.plot import plot_plotly
from plotly import graph_objs as go
import pandas as pd
import ta

In [ ]:
#Set start and end dates. The start date can be configured to be any date in the past, arbitrarily set to 2015-01-01. 
START = "2015-01-01"
TODAY = date.today().strftime("%Y-%m-%d")

In [ ]:

#Alpha Vantage API key
realtime_api_key = 'IDKN0O488FIDEPVR'

#News API (https://newsapi.org/) Key
news_api_key = '60d16a8cc922453896de9495f9b99b1d'

In [ ]:
# Function to check if the market is currently open
def is_market_open():
    now = datetime.now().time()
    market_open_time = time(9, 30)
    market_close_time = time(16, 0)
    return market_open_time <= now <= market_close_time

### API 1 - Real Time Stock Information

This function will retrieve the current stock information if the market is open using the Alpha Vantage API. If the market is closed, it will display a warning. 

In [ ]:
# Function to get real-time stock data

def get_real_time_stock_data(api_key, symbol):
    # Check if the stock market is open
    if is_market_open():
        try:
            # Construct the URL for the API request
            realtime_stock_url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval=5min&apikey={api_key}'
            
            # Send a GET request to the API
            response = requests.get(realtime_stock_url)
            
            # Parse the JSON response to a Python dictionary
            stock_data = response.json()
            
            # Return the 'Time Series (5min)' data if available, else return an empty dictionary
            return stock_data.get('Time Series (5min)', {})
        except Exception as e:
            # If any error occurs during the process, return None
            return None
    else:
        # If the market is not open, return None
        return None

### API 2 - Current News Topics

This function will search for the top news stories for a company using the News API (from the ticket symbol provided by the user) and display the top five stories. 

In [ ]:
# Define a function to fetch top news stories
def get_top_news(selected_stock):
    # Construct the URL for the news API request
    news_url = f'https://newsapi.org/v2/everything?q={selected_stock}&apiKey={news_api_key}'

    try:
        # Send a GET request to the news API
        response = requests.get(news_url)
        # Parse the JSON response to a Python dictionary
        news_data = response.json()

        # Check if the API request was successful and if there are any news articles
        if news_data.get('status') == 'ok' and news_data.get('totalResults') > 0:
            # Get the list of news articles
            articles = news_data['articles']

            # Display the top news stories
            for article in articles[:5]:  # Display the top 5 news stories
                st.markdown(f"**Title:** {article['title']}")
                st.write(f"**Description:** {article['description']}")
                st.markdown(f"**Source:** {article['source']['name']}")
                st.write(f"**Published At:** {article['publishedAt']}")
                st.write(f"**URL:** {article['url']}")
                st.write("---")
        else:
            # If there are no news articles, display a warning message
            st.warning("No news available for this stock.")

    except Exception as e:
        # If any error occurs during the process, display an error message
        st.error(f"An error occurred while fetching news: {e}")

### API 3 - Historical Stock Information

This function will retrieve and load the historical stock information using the Yahoo Finance API.

In [ ]:
# Load the stock data from Yahoo Finance
def load_stock_data(selected_stock):
    try:
        # Use yfinance to download the stock data from the start date to today
        data = yf.download(selected_stock, START, TODAY)
        # Reset the index of the DataFrame
        data.reset_index(inplace=True)
        # Return the DataFrame
        return data
    except Exception as e:
        # If any error occurs during the process, display an error message and return None
        st.error(f"An error occurred: {e}")
        return None

**Please note these code functions are taken from our main code project file "main.py". If you would like to run our stock engine, please refer to that file and follow the instructions at the top. This is because our front-end (GUI) does not run within Jupyter Notebook files.**

## Conclusion

There are several key takeaways that can benefit future students who take on similar work. This first is how to leverage and implement the resources of available APIs, specifically Yahoo Finance, Alpha Vantage, and News. Using APIs ended up being significantly simpler and easily repeatable for different stocks, giving us more flexibility in the ways we could implement our engine. We were able to switch to a just-in-time model as opposed to downloading information about thousands of stocks at once. One of the biggest takeaways was the importance of error handling which proved essential to a smooth extraction process with the possibility of network issues or API failures. We had several instances when the APIs behaved in ways that we didn't expect or that may have crashed our program. An example was our real-time stock API calls. When we originally coded that portion, it was after the market closed, and therefore we kept having errors with the API giving us no information. We discovered the API didn't know how to handle giving information when the market was closed, so we had to code a function to check first before sending the API calls. Finally, we learned about fetching textual data from external news sources. This included parsing the JSON response to extract the relevant information for users to stay informed about their selected stock. This was handled in a different way than our other two data sources and we had to parse the information out in order to display it in a suitable manner. 

Overall, we've learned a lot in this part of the project and are excited to continue developing. We have already built our frontend platform using Streamlit and have some base functionality set-up. We will continue to implement new features, add more buy/sell indicators, and improve our prediction accuracy.

## Documentation Statement

We extensively used the documentation for Yahoo Finance, Alpha Vantage, and News: 
https://pypi.org/project/yfinance/
https://www.alphavantage.co/documentation/
https://newsapi.org/docs


We used Streamlit's documentation in our frontend development: https://docs.streamlit.io/

We also used Investopedia to get the formulas for calulating the MACD, momentum, and RSI indicators: https://www.investopedia.com/terms/

Finally, we used Github Copilot to help comment our code functions. 